In [1]:
import numpy as np
import time

In [124]:
"""
Create sparse matrix
n = 5000
sparsity = 0.01
A = np.zeros((n,n))
for i in range(0,n):
    A[i,i] = i+1
# randn is normal distribution
A += sparsity*np.random.randn(n,n)
A = (A.T+A)/2
"""

'\nCreate sparse matrix\nn = 5000\nsparsity = 0.01\nA = np.zeros((n,n))\nfor i in range(0,n):\n    A[i,i] = i+1\n# randn is normal distribution\nA += sparsity*np.random.randn(n,n)\nA = (A.T+A)/2\n'

In [7]:
n = 4680
A = np.zeros((n,n))
with open('ne.hamil','r') as f:
    for line in f:
        tmpdat = (line.strip()).split()
        A[int(tmpdat[1])-1,int(tmpdat[0])-1] = A[int(tmpdat[0])-1,int(tmpdat[1])-1] = float(tmpdat[2])

In [3]:
"""
Define quantities needed in Davidson
"""
eigs = 4 # No. of eigenvectors to solve for
k = 2*eigs # No. of trial vectors is about double that of no. of eigs
t = np.eye(n,k) # 1000 by 8, so the columns are the trial unit vectors
V = np.zeros((n,n))
I = np.eye(n)
normconv = np.zeros(k)

In [4]:
"""
System parameters
"""
tol = 1e-3
maxiter = 100
maxbasis = k*16

In [14]:
"""
The main Davidson routine
"""
conv = False
start_davidson = time.time()
# first iteration
for j in range(0,k):
    V[:,j] = t[:,j]
nactive = k
theta_old = 1
while not conv:
    for m in range(1,maxiter+1):
        # 0-m is the subspace
        # V is the orthonormalised matrix, and R is discarded
        V[:,:nactive], R = np.linalg.qr(V[:,:nactive])
        
        # T = V^T A V
        tmp = np.dot(A,V[:,:nactive])
        T = np.dot(V[:,:nactive].T,tmp)
        t_eigval, t_eigv = np.linalg.eigh(T)
        # idx is an array of indices that would sort t_eigval from smallest to biggest
        # The BLAS dsyev already sorts them so no worries of having to write a sorting algo
        idx = t_eigval.argsort()
        theta = t_eigval[idx]
        s = t_eigv[:,idx]
        
        

        # Check for convergence
        norm = np.linalg.norm(theta[:eigs] - theta_old)
        print(f'iteration {m}, nactive: {nactive}, delta: {norm}')
        theta_old = theta[:eigs]
        print(theta[:eigs])
        print(np.diag(A)[:eigs])
        
        if nactive <= maxbasis-k:
            # Enlarge basis
            for j in range(0,k):
                # residue vectors
                tmpV = np.dot(V[:,:nactive],s[:,j])
                
                w = np.dot(A,tmpV)
                w -= theta[j]*tmpV
                normconv[j] = np.linalg.norm(w) < tol
                #vi print(np.linalg.norm(w))
                # correction vectors
                q = w/(theta[j]-A[j,j]+0.01)
                V[:,(nactive+j)] = q
            print(normconv.sum())
            if normconv.sum() == k:
                conv = True
                break
            nactive += k
        else:
            print('restarting')
            V[:,:k] = np.dot(V[:,:maxbasis],s[:,:k])
            #print(V[:,:k])
            #for j in range(0,k):
            #    # residue vectors
            #    w = np.dot((A-theta[j]*I),np.dot(V[:,:nactive],s[:,j]))
            #    normconv[j] = np.linalg.norm(w) < tol
            #    # correction vectors
            #    q = w/(theta[j]-A[j,j])
            #    V[:,(nactive+j)] = q
            #print(normconv.sum())
            #if normconv.sum() == k:
            #    conv = True
            #    break
            nactive = k
end_davidson = time.time()

iteration 1, nactive: 8, delta: 253.27734854495617
[-128.48877555 -126.59086954 -125.09720559 -122.29694662]
[-128.48877555  -94.64875923  -91.78612561  -91.78612561]
0.0
iteration 2, nactive: 16, delta: 0.32489143296655476
[-128.63393676 -126.7425256  -125.29579165 -122.44542787]
[-128.48877555  -94.64875923  -91.78612561  -91.78612561]
0.0
iteration 3, nactive: 24, delta: 0.07797624186392364
[-128.6585792  -126.77866753 -125.34657426 -122.48527673]
[-128.48877555  -94.64875923  -91.78612561  -91.78612561]
0.0
iteration 4, nactive: 32, delta: 0.042139997057978634
[-128.66674587 -126.79886887 -125.37815549 -122.50270133]
[-128.48877555  -94.64875923  -91.78612561  -91.78612561]
0.0
iteration 5, nactive: 40, delta: 1.790984148475471
[-128.67519996 -126.83693111 -125.46230693 -124.29128249]
[-128.48877555  -94.64875923  -91.78612561  -91.78612561]
0.0
iteration 6, nactive: 48, delta: 1.0102277141084186
[-128.67586148 -126.86551446 -125.88400384 -125.20884137]
[-128.48877555  -94.64875923

6.0
iteration 50, nactive: 16, delta: 6.614979620395744e-11
[-128.67639942 -127.03728377 -126.90754439 -126.90754439]
[-128.48877555  -94.64875923  -91.78612561  -91.78612561]
7.0
iteration 51, nactive: 24, delta: 6.366268937632734e-11
[-128.67639942 -127.03728377 -126.90754439 -126.90754439]
[-128.48877555  -94.64875923  -91.78612561  -91.78612561]
7.0
iteration 52, nactive: 32, delta: 5.4700575847857864e-11
[-128.67639942 -127.03728377 -126.90754439 -126.90754439]
[-128.48877555  -94.64875923  -91.78612561  -91.78612561]
7.0
iteration 53, nactive: 40, delta: 2.3034077151940243e-11
[-128.67639942 -127.03728377 -126.90754439 -126.90754439]
[-128.48877555  -94.64875923  -91.78612561  -91.78612561]
7.0
iteration 54, nactive: 48, delta: 2.3345489013300705e-11
[-128.67639942 -127.03728377 -126.90754439 -126.90754439]
[-128.48877555  -94.64875923  -91.78612561  -91.78612561]
7.0
iteration 55, nactive: 56, delta: 1.1498654060045368e-11
[-128.67639942 -127.03728377 -126.90754439 -126.90754439

(5000,)

In [80]:
print("davidson = ", theta[:eigs],";",
    end_davidson - start_davidson, "seconds")

# Begin Numpy diagonalization of A

start_numpy = time.time()

E,Vec = np.linalg.eig(A)
E = np.sort(E)

end_numpy = time.time()

# End of Numpy diagonalization. Print results.

print("numpy = ", E[:eigs],";",
     end_numpy - start_numpy, "seconds") 

davidson =  [0.99762663 1.99347973 3.01090566 3.98833208] ; -9080.740529298782 seconds
numpy =  [0.99757382 1.99341173 3.01083389 3.98827719] ; 56.50204348564148 seconds


In [116]:
A = np.zeros((2892,2892))
with open('rs2.hamil','r') as f:
    for line in f:
        tmpdat = (line.strip()).split()
        A[int(tmpdat[0])-1,int(tmpdat[1])-1] = A[int(tmpdat[1])-1,int(tmpdat[0])-1] = float(tmpdat[2])

In [10]:
A[:8,:8]

array([[-1.28488776e+02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -9.46487592e+01,  0.00000000e+00,
         0.00000000e+00,  2.68003788e-03,  0.00000000e+00,
         0.00000000e+00, -5.19717413e-02],
       [ 0.00000000e+00,  0.00000000e+00, -9.17861256e+01,
         0.00000000e+00,  0.00000000e+00,  1.34614735e-02,
         0.00000000e+00, -1.70792071e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -9.17861256e+01,  0.00000000e+00,  0.00000000e+00,
         1.34614735e-02,  0.00000000e+00],
       [ 0.00000000e+00,  2.68003788e-03,  0.00000000e+00,
         0.00000000e+00, -1.25097350e+02,  0.00000000e+00,
         0.00000000e+00,  1.47167803e-02],
       [ 0.00000000e+00,  0.00000000e+00,  1.34614735e-02,
         0.00000000e+00,  0.00000000e+00, -1.22296941e+02,
         0.00000000e+00, -3.55645487e-02],
       [ 0.00000000e+00,  0.000000